In [40]:
import tensorflow as tf
import pandas as pd
import numpy as np
import rethinkdb as r

In [41]:
conn = r.connect(
 host="localhost",
 port=28015,
 db='binance'
)

In [3]:
def get_last_time():
        # Get the last end time 
        last_event_time = r.table('features')\
            .max('epochEndTime')\
            .pluck('endTime')\
            .run(conn)

        last_time = (last_event_time['endTime']+1)/1000

        return last_time

def get_first_time():
    # Get the last end time 
    first_event_time = r.table('features')\
        .min('epochStartTime')\
        .pluck('startTime')\
        .run(conn)

    first_time = (first_event_time['startTime']+1)/1000

    return first_time

In [79]:
last_time = get_last_time()

# Derive the start time as a function of the 
# end time minus the window size
start = round(last_time - (int(100)*60))
index_columns = [
    'baseAsset',
    'startTime'
]

In [14]:
time_index = pd.to_datetime(list(range(start, round(last_time), 60)),unit='s')
time_index = time_index.round('min')

In [77]:
rec = r.table('features')\
.filter(lambda doc:
    r.expr(['BNB'])
        .contains(doc["baseAsset"])
)\
.pluck(['ask_price_0','ask_quantity_0', 'bid_price_0', 'bid_quantity_0', 'close', 'startTime'])\
.order_by(r.desc('startTime'))\
.limit(100)\
.run(conn)

In [62]:
pd.set_option('display.max_columns', 500)

In [51]:
for ret in rec:
    print(ret['ask_price_0'])
    print(ret['ask_quantity_0'])
    print('------------------------')

7.497e-05
0.34973505
------------------------
7.501e-05
0.5310708000000001
------------------------
7.499e-05
0.01364818
------------------------
7.5e-05
0.369
------------------------
7.496e-05
0.11528848
------------------------
7.498e-05
0.14156224
------------------------
7.498e-05
0.06868168000000001
------------------------
7.498e-05
0.00577346
------------------------
7.497e-05
0.29425724999999997
------------------------
7.495e-05
0.02645735
------------------------


In [84]:
# Get the klines for this period
rec = r.table('features')\
.between(
    r.epoch_time(start),
    r.epoch_time(last_time),
    index='epochEventTime'
)\
.pluck([
    'baseAsset',
    'startTime',
    'close',
    'open',
    'high',
    'low',
    'ask_price_0',
    'ask_price_1',
    'ask_price_2',
    'ask_price_3',
    'ask_price_4',
    'ask_quantity_0',
    'ask_quantity_1',
    'ask_quantity_2',
    'ask_quantity_3',
    'ask_quantity_4',
    'bid_price_0',
    'bid_price_1',
    'bid_price_2',
    'bid_price_3',
    'bid_price_4',
    'bid_quantity_0',
    'bid_quantity_1',
    'bid_quantity_2',
    'bid_quantity_3',
    'bid_quantity_4',
    'trades',
    'volume'
])\
.filter(lambda doc:
    r.expr(['BNB'])
        .contains(doc["baseAsset"])
)\
.run(conn)

f = pd.DataFrame(rec)
f['startTime'] = pd.to_datetime(f['startTime'], unit='ms')
f.set_index(index_columns, inplace=True)

# Reindex the dataframe based upon 
ind = pd.MultiIndex.from_product(
    [
        f.index.levels[0],
        time_index
    ],
    names=index_columns
)    
f = f.reindex(ind)

In [85]:
f

ask_price_0  ask_price_1  ask_price_2  \
baseAsset startTime                                                    
BNB       2019-03-29 13:17:00     0.004036     0.004036     0.004037   
          2019-03-29 13:18:00     0.004036     0.004036     0.004037   
          2019-03-29 13:19:00     0.004036     0.004036     0.004037   
          2019-03-29 13:20:00     0.004036     0.004036     0.004037   
          2019-03-29 13:21:00     0.004036     0.004036     0.004037   
          2019-03-29 13:22:00     0.004036     0.004036     0.004037   
          2019-03-29 13:23:00     0.004036     0.004036     0.004037   
          2019-03-29 13:24:00     0.004036     0.004036     0.004037   
          2019-03-29 13:25:00     0.004036     0.004036     0.004037   
          2019-03-29 13:26:00     0.004036     0.004036     0.004037   

                               ask_price_3  ask_price_4  ask_quantity_0  \
baseAsset startTime                                                       
BNB       2019-03-29 13:17:00     0.004037     0.004037        0.558031   
          2019-03-29 13:18:00     0.004037     0.004037        0.558031   
          2019-03-29 13:19:00     0.004037     0.004037        0.558031   
          2019-03-29 13:20:00     0.004037     0.004037        0.558031   
          2019-03-29 13:21:00     0.004037     0.004037        0.558031   
          2019-03-29 13:22:00     0.004037     0.004037        0.558031   
          2019-03-29 13:23:00     0.004037     0.004037        0.558031   
          2019-03-29 13:24:00     0.004037     0.004037        0.558031   
          2019-03-29 13:25:00     0.004037     0.004037        0.558031   
          2019-03-29 13:26:00     0.004037     0.004037        0.558031   

                               ask_quantity_1  ask_quantity_2  ask_quantity_3  \
baseAsset startTime                                                             
BNB       2019-03-29 13:17:00          0.2002        0.999108        1.147045   
          2019-03-29 13:18:00          0.2002        0.999108        1.147045   
          2019-03-29 13:19:00          0.2002        0.999108        1.147045   
          2019-03-29 13:20:00          0.2002        0.999108        1.147045   
          2019-03-29 13:21:00          0.2002        0.999108        1.147045   
          2019-03-29 13:22:00          0.2002        0.999108        1.147045   
          2019-03-29 13:23:00          0.2002        0.999108        1.147045   
          2019-03-29 13:24:00          0.2002        0.999108        1.147045   
          2019-03-29 13:25:00          0.2002        0.999108        1.147045   
          2019-03-29 13:26:00          0.2002        0.999108        1.147045   

                               ask_quantity_4  bid_price_0  bid_price_1  \
baseAsset startTime                                                       
BNB       2019-03-29 13:17:00        0.363936     0.004033     0.004033   
          2019-03-29 13:18:00        0.363936     0.004033     0.004033   
          2019-03-29 13:19:00        0.363936     0.004033     0.004033   
          2019-03-29 13:20:00        0.363936     0.004033     0.004033   
          2019-03-29 13:21:00        0.363936     0.004033     0.004033   
          2019-03-29 13:22:00        0.363936     0.004033     0.004033   
          2019-03-29 13:23:00        0.363936     0.004033     0.004033   
          2019-03-29 13:24:00        0.363936     0.004033     0.004033   
          2019-03-29 13:25:00        0.363936     0.004033     0.004033   
          2019-03-29 13:26:00        0.363936     0.004033     0.004033   

                               bid_price_2  bid_price_3  bid_price_4  \
baseAsset startTime                                                    
BNB       2019-03-29 13:17:00     0.004032     0.004032     0.004032   
          2019-03-29 13:18:00     0.004032     0.004032     0.004032   
          2019-03-29 13:19:00     0.004032     0.004032     0.004032   
          2019-03-29 13:20:00  

In [ ]:
f=f.fillna(axis=0, method="ffill")\
           .fillna(axis=0, method="bfill")

In [ ]:
x = -tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits=policy_logits, 
      labels=actions
)